In [6]:
import numpy as np
from numba import njit
from scipy import sparse
from tqdm import tqdm

In [7]:
@njit
def gen_net(N, p):
    acc = (np.random.rand(N, N)+np.diag(np.ones(N)))<p
    res = np.where(acc>0)
        
    return np.stack(res, axis=-1)

In [8]:
@njit
def f(m, N, p, q):
    s = np.zeros(N)
    s[np.random.choice(N, m, replace=False)] = 1
    net = gen_net(N, p)
    step = 0

    while True:
        acc = np.zeros(N)
        for line in net:
            if s[line[0]]==1:
                acc[line[1]] += 1

        finish = True
        for i in range(N):
            if s[i]==1:
                continue
            elif acc[i]>=q:
                s[i] = 1
                finish = False
        
        step += 1
        if finish:
            break

    if s.sum()/N > 0.98:
        return 1, step
    else:
        return 0, step

In [16]:
def Np_rel(m, q):
    sample = 5
    Iter = 500
    Npq = 0.2

    ps = np.linspace(0.03, 0.05, sample)
    res = np.zeros(sample)
    for i in range(sample):
        p = ps[i]
        print(f"p={p}")
        for j in tqdm(range(Iter)):
            res[i] += f(m, np.round(Npq/p**q).astype(int), p, q)[0]
        
    res /= Iter

    return res

In [15]:
%timeit f(5, 10000, 0.01, 3)

955 ms ± 74.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def pq_rel(m, q):
    sample = 5
    Iter = 3000
    Npq = 0.0625
    N = 500

    qs = np.arange(1, 6)
    res = np.zeros(sample)
    for i in range(sample):
        q = qs[i]
        print(f"q={q}")
        for j in tqdm(range(Iter)):
            res[i] += f(m, N, (Npq/N)**(1/q), q)[0]
        
    res /= Iter

    return res

In [17]:
b = Np_rel(5, 3)

p=0.03


100%|██████████| 500/500 [05:43<00:00,  1.45it/s]


p=0.035


100%|██████████| 500/500 [02:06<00:00,  3.95it/s]


p=0.04


100%|██████████| 500/500 [00:58<00:00,  8.50it/s]


p=0.045


100%|██████████| 500/500 [00:28<00:00, 17.59it/s]


p=0.05


100%|██████████| 500/500 [00:14<00:00, 33.45it/s]


In [18]:
b

array([0.816, 0.756, 0.798, 0.762, 0.778])

In [12]:
a = Np_rel(5, 3)

p=0.03


100%|██████████| 3000/3000 [02:44<00:00, 18.21it/s]


p=0.035


100%|██████████| 3000/3000 [01:09<00:00, 43.37it/s]


p=0.04


100%|██████████| 3000/3000 [00:26<00:00, 112.92it/s]


p=0.045


100%|██████████| 3000/3000 [00:14<00:00, 209.71it/s]


p=0.05


100%|██████████| 3000/3000 [00:08<00:00, 374.57it/s]


In [13]:
a

array([0.14333333, 0.13866667, 0.13166667, 0.12133333, 0.108     ])